# 1. Gesture Recognition
In this group project, you are going to build a 3D Conv model that will be able to predict the 5 gestures correctly. Please import the following libraries to get started.

In [ ]:
import numpy as np
import os

from skimage.transform import resize as imresize
import imageio as imio
import datetime
import cv2

import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

We set the random seed so that the results don't vary drastically.

In [ ]:
np.random.seed(30)
import random as rn
rn.seed(30)
from keras import backend as K
import tensorflow as tf
tf.set_random_seed(30)

The default path written in the starter code threw error. We use the following cell to get the location , where we are running the notebook.

In [ ]:
os.getcwd()

In [ ]:
train_path = '/mnt/disks/user/project/PROJECT/Project_data/train'
val_path = '/mnt/disks/user/project/PROJECT/Project_data/val'

# 2. Pre-Processing

Since the image size is not uniform across all the folders, let us figure out the available image sizes. We loop through each training folder and read the shape of the size of the first image in each of them.<br>
We keep appending the sizes in a list called __image_size__ and print the result at the end.

In [ ]:
# List all folders in the train_path

train_folder=os.listdir(train_path)

# Create a blank list
image_size=[]

# Loop through each folder and read the 1st image.
# Read in its shape and append it to the list image_size, if it
# already doesn't exist.

for folder in train_folder:
    
    file_path=train_path+'/'+folder
    fst_image=os.listdir(train_path+'/'+folder)[0]
    img=imio.imread(file_path+'/'+fst_image)
    if img.shape not in image_size:
        image_size.append(img.shape)

print('The available Sizes Are')
print(image_size)

So , we've established that we've only two sizes. We will use __80x80__ as our reference to resize all images.<br> We initiate the important iteration parameters in the cell below.

In [ ]:
train_doc = np.random.permutation(open('/mnt/disks/user/project/PROJECT/Project_data/train.csv').readlines())
val_doc = np.random.permutation(open('/mnt/disks/user/project/PROJECT/Project_data/val.csv').readlines())

#experiment with the batch size
batch_size = 32
ht=80
wd=80
num_epochs =60

In this block, you read the folder names for training and validation. You also set the `batch_size` here. Note that you set the batch size in such a way that you are able to use the GPU in full capacity. You keep increasing the batch size until the machine throws an error.

### 2.1 Motion Detection

After numerous futile methods , we tried to figure out what was common to all our models. We realized that the images are of poor quality and were taken at a typical workplace. That's why the images were plagued with many bright spots.<br> We then decided to explore feature extraction. In all the videos, the individual images weren't important. Rather the difference between each consecutive image was the key feature.<br> Thus, we decided to feed the difference between consecutive images into the model-fit methos. <br> We demonstrate an example below in the following two cells to do the following<br>

- Select a folder at random from __train_doc__
- Read in all the image file names into a list called __images__.
- Iterate through the 30 images and calculate the difference between each image.
- Invert the difference and plot.
- In the end , we will have 29 difference images.
- In some difference plots, we see all blank. This means there was no movement between two consecutive images.
- There are 29 differences, so last subplot is empty.

__This is the key EDA we did to eliminate all irrelevant information from the training data. We will feed only the 29 difference images into the model for training__ 

In [ ]:
t = np.random.permutation(train_doc)
path='/mnt/disks/user/project/PROJECT/Project_data/train/'+t[0].split(';')[0]
images=os.listdir(path)

In [ ]:
f, ax = plt.subplots(5, 6, sharey=True,sharex=True)
f.set_figwidth(15)
f.set_figheight(15)

images.sort()
print(t[0].split(';')[1])
for r in range(0,5):
    for c in range(0,6):
        i=c+r*6
        if i<29:
            
            # Read the ith and i+1th image and resize them to 80x80
            path1=path+'/'+images[i]
            path2=path+'/'+images[i+1]
            
            img1=cv2.imread(path1).astype(np.float32)
            img1=imresize(img1,(80,80))
            
            img2=cv2.imread(path2).astype(np.float32)
            img2=imresize(img2,(80,80))
            
            # Calculate the absolute difference between two consecutive images
            img3=cv2.absdiff(img1,img2)
            imagem = abs(img3 - 255)
            ax[r,c].imshow(imagem/255)        

### 2.2 Generator

This is one of the most important part of the code. The overall structure of the generator has been given. In the generator, you are going to preprocess the images as you have images of 2 different dimensions as well as create a batch of video frames. You have to experiment with `img_idx`, `y`,`z` and normalization such that you get high accuracy.<br>

The generator code functioning is described below:

- We've 663 folders in training dataset, with a batch-size of 32.
- This means, in one run, we'll have 663/32=20 ( integer div ) folders in each batch.
- After 32 batches, we would cover 32x20=640 folders and 23 folders would still remain.
- We address this issue by inserting these remaining folders into the last batch of a single run.
- The first __for loop__ has __batch__ iterating from 0 to 32 , as our __num_bathes=32__
- The second __for loop__ has __folder__ iterating from 0 to 19.
- When we read in the images from a folder, we calculate the difference between each consecutive image and feed it into the batch.

In [ ]:
def generator(source_path, folder_list, batch_size):
    
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    
    # create a list of image numbers you want to use for a particular video
    # Instead of feeding every image into the training process, we produce 29 differences.
    
    img_idx = range(0,29)
    
    # The number of images per sequence is determined by the length of img_idx
    
    x=len(img_idx)
    
    # We'll use these dimensions for our resizing the images , as established above.
    ht=80
    wd=80
   
    while True:
        
        # t is the folder list but fully randomised
        t = np.random.permutation(folder_list)
        
        # calculate the number of batches
        num_batches =len(t)//batch_size
        
        # we iterate over the number of batches from 0 to 19 ( for batch_size = 32 )
        for batch in range(num_batches):
            
            batch_data = np.zeros((batch_size,x,ht,wd,3))
            batch_labels = np.zeros((batch_size,5))
            # folder iterates from 0 to 31 for a batch size of 32.
            for folder in range(batch_size):
                
                # i is an index to iterate through the randomised folder list t
                i=folder+(batch*batch_size)
                
                # list all images in the folder identified by the index i
                imgs = os.listdir(source_path+'/'+ t[i].split(';')[0])
                
                # The image file list in imgs above aren't necessarily sorted. There is a danger of
                # losing the temporal information. That's why we sort the list below.
                
                imgs.sort()
                
                # Iterate iver the frames/images of a folder to read them in
                
                for idx,item in enumerate(img_idx): 
                    img1 = cv2.imread(source_path+'/'+ t[i].strip().split(';')[0]+'/'+imgs[idx]).astype(np.float32)
                    img2 = cv2.imread(source_path+'/'+ t[i].strip().split(';')[0]+'/'+imgs[idx+1]).astype(np.float32)
                    
                    # The key idea is to capture motion from the images. Instead of reading 30 images,
                    # we calculate the difference between consecutive images and feed it into the model.
                    
                    diff_image=cv2.absdiff(img1,img2)
                    image = abs(diff_image - 255)/255
                    
                    image = imresize(image, (ht, wd))
                     
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    
                    batch_data[folder,idx,:,:,0] = image[:, :, 0]
                    batch_data[folder,idx,:,:,1] = image[:, :, 1]
                    batch_data[folder,idx,:,:,2] = image[:, :, 2]
                    
            batch_labels[folder, int(t[i].strip().split(';')[2])] = 1    
            yield batch_data,batch_labels
        
        # We now make a batch of the remaining folders.
        # the variable i has the last folder index we squeezed into the batch.
        # Incrementing it by 1 , will point to the next folder for the new batch.
        
        rem_folders=len(t)-(i+1)
        batch_labels = np.zeros((rem_folders,5))
        batch_data = np.zeros((rem_folders,x,ht,wd,3))
        
        # m is counter for iterating through the remaining folders  
        for m in range(0,rem_folders):
            
            # m+i+1 is now an index pointing to folders in t, that were left above.
            imgs = os.listdir(source_path+'/'+ t[m+i+1].split(';')[0]) # read all the images in the folder
            imgs.sort()
            for idx,item in enumerate(img_idx): #  Iterate over the frames/images of a folder to read them in
                
                img1 = cv2.imread(source_path+'/'+ t[m+i+1].strip().split(';')[0]+'/'+imgs[idx]).astype(np.float32)
                img2 = cv2.imread(source_path+'/'+ t[m+i+1].strip().split(';')[0]+'/'+imgs[idx+1]).astype(np.float32)
                
                diff_image=cv2.absdiff(img1,img2)
                
                # We normalize here instead of the individual channels 
                image = abs(diff_image - 255)/255
                
                image = imresize(image, (ht, wd))
                batch_data[m,idx,:,:,0] = image[:, :, 0]
                batch_data[m,idx,:,:,1] = image[:, :, 1]
                batch_data[m,idx,:,:,2] = image[:, :, 2]
            
            batch_labels[m, int(t[m+i+1].strip().split(';')[2])] = 1
        
        yield batch_data,batch_labels

Note here that a video is represented above in the generator as (number of images, height, width, number of channels). Take this into consideration while creating the model architecture.

In [ ]:
curr_dt_time = datetime.datetime.now()

num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)

# choose the number of epochs

print ('# epochs =', num_epochs)

### 2.3 Testing the Generator Code

We test the generator code by using a compact training set of 15 folders and a batch size of 6.<br>
This means , the generator code should give us three batches of 6,6 and 3 folders each in one run. We simulate this reading in only the 1st 15 folders into __small_train_doc__ and calling the genrator thrice.

In [ ]:
small_train_doc = train_doc[0:15]
x=generator(train_path, small_train_doc, 6)

batch_data,batch_labels=next(x)[:]
print(batch_data.shape,batch_labels.shape)
batch_data,batch_labels=next(x)[:]
print(batch_data.shape,batch_labels.shape)
batch_data,batch_labels=next(x)[:]
print(batch_data.shape,batch_labels.shape)
print('Success !!!')

In [ ]:
from keras.models import Sequential, Model
from keras.layers import Dense, GRU, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation,Dropout,LSTM
from keras.layers.convolutional import Conv2D, MaxPooling2D,Conv3D,MaxPooling3D
from keras.callbacks import ModelCheckpoint,ReduceLROnPlateau
from keras.layers.normalization import BatchNormalization
from keras import optimizers
import pandas as pd

## 3. Model Building
Here you make the model using different functionalities that Keras provides. Remember to use `Conv3D` and `MaxPooling3D` and not `Conv2D` and `Maxpooling2D` for a 3D convolution model. You would want to use `TimeDistributed` while building a Conv2D + RNN model. Also remember that the last layer is the softmax. Design the network in such a way that the model is able to give good accuracy on the least number of parameters so that it can fit in the memory of the webcam.

In the model building section, we build 3 models.

- Conv2D+GRU Model Runnning on pre-processed images with Adam Optmizer
- Conv3D Model Runnning on pre-processed images with Adam Optmizer
- Conv3D Model Running __without__ pre-processed images with SGD Optmizer __( This is our final Submission )__

<span style="color:red"> __Final Submitted Model is from Section 3.3 Conv3D. Kindly jump to Section 3.3__ </span>

### 3.1 Three Conv2D Layers Followed by One GRU

In [ ]:
model = Sequential()

# Layer 1
model.add(TimeDistributed(Conv2D(32,(3,3),padding='same',activation='relu'), input_shape=(29, ht, wd, 3)))
model.add(TimeDistributed(Conv2D(32,(3,3),activation='relu')))
model.add(TimeDistributed(MaxPooling2D((2,2))))

# Layer 2
model.add(TimeDistributed(Conv2D(64,(3,3),padding='same',activation='relu')))
model.add(TimeDistributed(MaxPooling2D((2,2))))

# Layer 3
model.add(TimeDistributed(Conv2D(128,(3,3),padding='same',activation='relu')))
#model.add(TimeDistributed(BatchNormalization()))
model.add(TimeDistributed(MaxPooling2D((2,2))))

model.add(TimeDistributed(Flatten()))

# Entering GRU RNN
model.add(GRU(256, activation='relu', return_sequences=False))
model.add(Dense(256, activation='relu'))
model.add(Dropout(.5))
model.add(Dense(5, activation='softmax'))

Now that you have written the model, the next step is to `compile` the model. When you print the `summary` of the model, you'll see the total number of parameters you have to train.

In [ ]:
optimiser = optimizers.Adam(lr=0.001)
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())

Let us create the `train_generator` and the `val_generator` which will be used in `.fit_generator`.

In [ ]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [ ]:
model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

#LR = # write the REducelronplateau code here
callbacks_list = [checkpoint]

The `steps_per_epoch` and `validation_steps` are used by `fit_generator` to decide the number of next() calls it need to make.

In [ ]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

Let us now fit the model. This will start training the model and with the help of the checkpoints, you'll be able to save the model at the end of each epoch.

In [ ]:
history=model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list,validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Let us call the __history__ output to visualize the results

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2)
fig.set_figwidth(20)
fig.set_figheight(5)

fig.suptitle('Model 1 Performance')
ax1.plot(history.history['loss'],label='Training')
ax1.plot(history.history['val_loss'],label='Validation')
ax1.set_title('Loss')
ax1.set_ylim(0, 0.2)
ax1.legend(loc="upper right")

ax2.plot(history.history['categorical_accuracy'],label='Training')
ax2.plot(history.history['val_categorical_accuracy'],label='Validation')
ax2.set_title('Accuracy')
ax2.set_ylim(0.0, 1)
ax2.legend(loc="bottom right")

In [ ]:
import pandas as pd
pd.DataFrame({ key: pd.Series(val) for key, val in history.history.items()}).to_csv('Model with Pre-processing 28th Feb 60 Epochs.csv')

In [ ]:
model.save("Conv2D 60 Epochs 29th Feb.h5")

The Conv2D+GRU Model seems to be converging well after Epoch 40, but cost remains surprisingly steady !!

We will now attempt a Conv3D model. Since the pre-processed images were very successful in the CNN+GRU Model, we'll continue to use them.<br>
We'll make a minor change here in the batch size and make it 15 instead of 32 to deal with __kernel died__ problem. The model layers remain exactly same. <br>
However, instead of __Time Distributed__ we use __Conv3D__ here. This implies changing the Kernels to 3D as well. 

### 3.2 Conv3D on Pre-Processed Images

This model will take the difference images as the Conv2D Model. Otherwise all other steps remain same.

In [ ]:
model2 = Sequential()

# Layer 1
model2.add(Conv3D(32, (3,3,3), activation='relu', input_shape=(29, ht, wd, 3)))
model2.add(Conv3D(32, (3,3,3), activation='relu'))
model2.add(MaxPooling3D(pool_size=(1, 2, 2)))

# Layer 2
model2.add(Conv3D(64, (3,3,3), activation='relu'))
model2.add(MaxPooling3D(pool_size=(1, 2, 2)))

# Layer 3
model2.add(Conv3D(128, (3,3,3), activation='relu'))
model2.add(MaxPooling3D(pool_size=(1, 2, 2)))
 
# Layer 4
model2.add(Flatten())
model2.add(Dense(256))
model2.add(Dropout(0.5))

model2.add(Dense(5, activation='softmax'))

In [ ]:
optimiser = optimizers.Adam(lr=0.001)
model2.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model2.summary())

The model remains shallow, but the parameter count has reached 4 Million. In the cell below, we change the batch size and initialize the generators again.

In [ ]:
batch_size2=15
train_generator = generator(train_path, train_doc, batch_size2)
val_generator = generator(val_path, val_doc, batch_size2)

In [ ]:
model_name = 'model_init_3D' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)

#LR = # write the REducelronplateau code here
callbacks_list = [checkpoint]

In [ ]:
if (num_train_sequences%batch_size2) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size2)
else:
    steps_per_epoch = (num_train_sequences//batch_size2) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size2)
else:
    validation_steps = (num_val_sequences//batch_size2) + 1

In [ ]:
checkpoint2 = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True,
                              save_weights_only=False, mode='auto', period=1)
callbacks_list2 = [checkpoint2]
history2=model2.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list2,validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

In [ ]:
fig3, (ax1, ax2) = plt.subplots(1, 2)
fig3.set_figwidth(20)
fig3.set_figheight(5)

fig3.suptitle('Model 2 Conv 3D Performance')
ax1.plot(history2.history['loss'],label='Training')
ax1.plot(history2.history['val_loss'],label='Validation')
ax1.set_title('Loss')
ax1.set_ylim(0, 3)
ax1.legend(loc="upper right")

ax2.plot(history2.history['categorical_accuracy'],label='Training')
ax2.plot(history2.history['val_categorical_accuracy'],label='Validation')
ax2.set_title('Accuracy')
ax2.set_ylim(0.4, 1)
ax2.legend(loc="bottom right")

In [ ]:
import pandas as pd
pd.DataFrame({ key: pd.Series(val) for key, val in history2.history.items()}).to_csv('Model2 Shallow with Pre-processing 28th Feb Conv3D.csv')

In [ ]:
model2.save("Conv3D 60 Epochs.h5")

<span style="color:red">Although the peformance of the __Time Distributed Conv2D__ and the __Conv3D__ models were very good on the pre-processed images, we cannot use them for submission. This is because, we do not have access to test data. Without having the test data-set, we are not sure , how to apply pr-processing on test data.<br></span>

<span style="color:red">Therefore , we build the final model below __Conv3D Without__ Pre-processing.</span>

### 3.3 Conv3D Without Pre-processing

This is the final Conv3D version. Here, we will use the images as they are. However, the following changes will be implemented.

- The Generator function at the top was subtracting Nth image from N+1th image and inserting 29 difference images into the batch. __We will not do that here__.
- In this case, we will keep the images as they are. However, we'll take every altertae image from each frame. __Our Image count will be thus 15 per frame__. Thus we define another __generator2__.
- We will increase the epochs to __80__ giving the model more time to converge.
- Our layer structure becomes deeper than the previous variant but parameter count is lesses.
- Input shape now becomes __(15,80,80,3)__ instead of __(29,80,80,3)__
- Instead of Adam, we use SGD with Nesterov Momentum for optimizer. We also implement __Reduce Learning Rate on Plateau__ in callback.

In [ ]:
def generator2(source_path, folder_list, batch_size):
    
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    img_idx = range(0,30,2)
    x=len(img_idx)
    ht=80
    wd=80
    while True:
        
        t = np.random.permutation(folder_list)
        num_batches =len(t)//batch_size
        for batch in range(num_batches):
            
            batch_data = np.zeros((batch_size,x,ht,wd,3))
            batch_labels = np.zeros((batch_size,5))
            
            for folder in range(batch_size):
                i=folder+(batch*batch_size)
                imgs = os.listdir(source_path+'/'+ t[i].split(';')[0])
                imgs.sort()
                for idx,item in enumerate(img_idx): 
                    img1 = cv2.imread(source_path+'/'+ t[i].strip().split(';')[0]+'/'+imgs[idx]).astype(np.float32)
                    image = imresize(img1, (ht, wd))
                    batch_data[folder,idx,:,:,0] = image[:, :, 0]/255
                    batch_data[folder,idx,:,:,1] = image[:, :, 1]/255
                    batch_data[folder,idx,:,:,2] = image[:, :, 2]/255
                    
            batch_labels[folder, int(t[i].strip().split(';')[2])] = 1    
            yield batch_data,batch_labels
        
        rem_folders=len(t)-(i+1)
        batch_labels = np.zeros((rem_folders,5))
        batch_data = np.zeros((rem_folders,x,ht,wd,3))
        for m in range(0,rem_folders):
            
            imgs = os.listdir(source_path+'/'+ t[m+i+1].split(';')[0])
            imgs.sort()
            for idx,item in enumerate(img_idx):
                
                img1 = cv2.imread(source_path+'/'+ t[m+i+1].strip().split(';')[0]+'/'+imgs[idx]).astype(np.float32)
                image = imresize(img1, (ht, wd))
                batch_data[m,idx,:,:,0] = image[:, :, 0]/255
                batch_data[m,idx,:,:,1] = image[:, :, 1]/255
                batch_data[m,idx,:,:,2] = image[:, :, 2]/255
            
            batch_labels[m, int(t[m+i+1].strip().split(';')[2])] = 1
        
        yield batch_data,batch_labels

In [ ]:
model3 = Sequential()

# Layer 1
model3.add(Conv3D(32, (3,3,3), activation='relu', input_shape=(15, 80, 80, 3)))
model3.add(MaxPooling3D(pool_size=(1, 2, 2), strides=(1, 2, 2)))

# Layer 2
model3.add(Conv3D(64, (3,3,3), activation='relu'))
model3.add(MaxPooling3D(pool_size=(1, 2, 2), strides=(1, 2, 2)))

# Layer 3
model3.add(Conv3D(128, (3,3,3), activation='relu'))
model3.add(Conv3D(128, (3,3,3), activation='relu'))
model3.add(MaxPooling3D(pool_size=(1, 2, 2), strides=(1, 2, 2)))

# Layer 4
model3.add(Conv3D(256, (2,2,2), activation='relu'))
model3.add(Conv3D(256, (2,2,2), activation='relu'))
model3.add(MaxPooling3D(pool_size=(1, 2, 2), strides=(1, 2, 2)))

# Layer 5
model3.add(Flatten())
model3.add(Dense(1024))
model3.add(Dropout(0.5))
model3.add(Dense(1024))
model3.add(Dropout(0.5))
model3.add(Dense(5, activation='softmax'))

In [ ]:
optimiser = optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model3.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model3.summary())

In [ ]:
batch_size2=15
num_epochs2 =80
train_generator = generator2(train_path, train_doc, batch_size2)
val_generator = generator2(val_path, val_doc, batch_size2)

In [ ]:
model_name = 'model_init_3D_no_pre_process' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

In [ ]:
if (num_train_sequences%batch_size2) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size2)
else:
    steps_per_epoch = (num_train_sequences//batch_size2) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size2)
else:
    validation_steps = (num_val_sequences//batch_size2) + 1

In the previous two models, we used Adam optimiser, so we did not implement Reduce LR on Plateau. Here we switch to an SGD Optimiser.

In [ ]:
checkpoint3 = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False,
                              save_weights_only=True, mode='auto', period=1)

LR =ReduceLROnPlateau(monitor='loss',factor=0.1,patience=2,verbose=0,mode='min',epsilon=0.001,cooldown=0,min_lr=0)
callbacks_list2 = [checkpoint3,LR]

history3=model3.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs2, verbose=1, 
                    callbacks=callbacks_list2,validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

In [ ]:
fig4, (ax1, ax2) = plt.subplots(1, 2)
fig4.set_figwidth(20)
fig4.set_figheight(5)

fig4.suptitle('Model 3 Conv 3D Performance No Pre Processing')
ax1.plot(history3.history['loss'],label='Training')
ax1.plot(history3.history['val_loss'],label='Validation')
ax1.set_title('Loss')
ax1.set_ylim(0, 0.5)
ax1.legend(loc="upper right")

ax2.plot(history3.history['categorical_accuracy'],label='Training')
ax2.plot(history3.history['val_categorical_accuracy'],label='Validation')
ax2.set_title('Accuracy')
ax2.set_ylim(0.0, 1)
ax2.legend(loc="bottom right")

We notice that , Epochs between __10 to 80__ are well fitting the validation data. We will pick our submission model from one of these Epochs.<br>
Once again, our loss remains static. 